In [43]:
import os
import cv2

img = cv2.imread(os.path.join('..', 'assets', 'B&W_birds.jpg'))
cv2.imshow('Original Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [44]:
grey_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret , thresh =cv2.threshold(grey_img, 127, 255, cv2.THRESH_BINARY_INV) # ret is the threshold value used, here 127, thresh is the thresholded image
cv2.imshow('Thresholded Image', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [45]:
contours,hierarchy = cv2.findContours(thresh, 1, 2) # 1 means cv2.RETR_LIST , 2 means cv2.CHAIN_APPROX_SIMPLE

Here is the very brief breakdown of those two flags:

* **`cv2.RETR_LIST` (Retrieval Mode):** This tells OpenCV to find **all** contours but **ignore any hierarchy** (relationship). It gives you a simple, flat list of every single shape found, without caring if one shape is inside another (like a nested box).
* **`cv2.CHAIN_APPROX_SIMPLE` (Approximation Method):** This is a memory-saving trick. Instead of storing every single pixel along a straight line, it only stores the **endpoints**.
* *Example:* For a rectangle, instead of saving hundreds of points along the edges, it saves just the **4 corner points**.

---

In [46]:
for cnt in contours:
    print("Contour Area: ", cv2.contourArea(cnt))
    
    # all the small contours will be noise, we can filter them out
    if cv2.contourArea(cnt) > 1000:
        # cv2.drawContours(img, [cnt], 0, (255,0,0), 3) # draw only the filtered contours
        x1, y1, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(img, (x1, y1), (x1+w, y1+h), (255,0,0), 3)

cv2.imshow('Filtered Contours', img)
cv2.waitKey(0)
cv2.destroyAllWindows()          

Contour Area:  13.5
Contour Area:  37.0
Contour Area:  4.0
Contour Area:  2.0
Contour Area:  7.0
Contour Area:  2.0
Contour Area:  2.0
Contour Area:  2.0
Contour Area:  4.0
Contour Area:  2.0
Contour Area:  2.0
Contour Area:  5263.5
Contour Area:  2.0
Contour Area:  2.0
Contour Area:  2.0
Contour Area:  6.0
Contour Area:  5124.0
Contour Area:  18.5
Contour Area:  3570.0
Contour Area:  2.0
Contour Area:  14.0
Contour Area:  5069.0
Contour Area:  2.0
Contour Area:  5387.0


In [47]:
cnt = contours[0]
print("Number of contours found = ", len(contours))
print("First contour = ", cnt)
cv2.drawContours(img, contours, -1, (0,255,0), 3) # -1 means draw all contours
cv2.imshow('Contours', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Number of contours found =  24
First contour =  [[[ 76 470]]

 [[ 77 469]]

 [[ 79 469]]

 [[ 80 470]]

 [[ 78 472]]

 [[ 76 472]]

 [[ 75 473]]

 [[ 74 472]]

 [[ 74 471]]

 [[ 75 470]]]


#### **Optional**


In [ ]:
# *Optional*
for cnt in contours:
    approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
    print("Number of vertices = ", len(approx))
    if len(approx) == 3:
        shape_name = "Triangle"
    elif len(approx) == 4:
        shape_name = "Quadrilateral"
    elif len(approx) == 5:
        shape_name = "Pentagon"
    elif len(approx) == 6:
        shape_name = "Hexagon"
    else:
        shape_name = "Circle"
    
    M = cv2.moments(cnt)
    if M['m00'] != 0:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
    else:
        cx, cy = 0, 0

    cv2.putText(img, shape_name, (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)   
    
    
cv2.imshow('Shapes Detected', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Number of vertices =  10
Number of vertices =  12
Number of vertices =  6
Number of vertices =  4
Number of vertices =  6
Number of vertices =  4
Number of vertices =  4
Number of vertices =  4
Number of vertices =  6
Number of vertices =  4
Number of vertices =  4
Number of vertices =  15
Number of vertices =  4
Number of vertices =  4
Number of vertices =  4
Number of vertices =  6
Number of vertices =  13
Number of vertices =  14
Number of vertices =  17
Number of vertices =  4
Number of vertices =  15
Number of vertices =  16
Number of vertices =  4
Number of vertices =  15


---
### **Refrences:**

https://docs.opencv.org/4.x/d3/d05/tutorial_py_table_of_contents_contours.html

---

# **Notes**

## Core Concepts

* **What are Contours?** Contours are curves connecting all continuous points (along the boundary) that have the same color or intensity. They essentially represent the borders of shapes.
* **The Goal:** In this lesson, the instructor demonstrates how to build a functional **object detector** (specifically to detect birds in the sky) using basic image processing instead of complex AI models like YOLO.

## The Process: From Image to Detection

To detect contours, you must prepare the image first:

1. **Grayscale Conversion:** You must convert the image to grayscale first (`cv2.cvtColor`) because thresholding requires a single-channel image.
2. **Inverse Binary Thresholding:**
    * Contours work best on **binary images** (black and white).
    * The instructor used **`cv2.THRESH_BINARY_INV`** because the birds were dark and the sky was light. Contours detect **white objects** on a black background, so the dark birds needed to be "flipped" to white.


3. **Finding Contours (`cv2.findContours`):**
    * This function scans the binary image and returns a list of all detected contours.
    * **Note:** Depending on your OpenCV version, this function might return 2 values (contours, hierarchy) or 3 values (image, contours, hierarchy).



## Filtering and Visualization

* **Noise Filtering (Area Check):**
    * `cv2.findContours` detects *everything*, including tiny specs of noise.
    * The instructor used `cv2.contourArea(cnt)` to calculate the size of each contour and ignored any contour smaller than a specific threshold (e.g., 200 pixels).


* **Drawing Contours (`cv2.drawContours`):**
    * This function draws the outline directly onto the image.
    * **Key Argument:** You must pass **`-1`** as the third argument to draw *all* contours in the list (or the index of a specific contour to draw just one).



## Building the Object Detector (Bounding Boxes)

Instead of just drawing the outline, the instructor created a standard detection box:

1. **Get Bounding Rectangle:**
    * Used `x, y, w, h = cv2.boundingRect(cnt)` to get the coordinates of a rectangle that perfectly encloses the contour.


2. **Draw the Rectangle:**
    * Used standard `cv2.rectangle` logic with the calculated coordinates to draw boxes around every bird.


---
---

# **How Contours differs from edges?**

Think of a **Contour** as the **outline** or the **silhouette** of an object.

Here is the simplest way to visualize it:

### 1. The "Island" Analogy

Imagine your image is an ocean (black background). In this ocean, there are islands (white objects).

* The **Contour** is the **coastline**.
* It is the continuous line that separates the "land" (object) from the "water" (background).

### 2. Why is it different from "Edges"?

This is the most common point of confusion.

* **Edges (Canny, Sobel):** These are just "dumb" pixels where the color changes sharply. They are like unconnected dots or sketched lines. If you have a gap in the line, the computer doesn't know it's a circle; it just sees curved lines.
* **Contours:** These are **mathematical shapes**. OpenCV understands that these points belong together as a single, closed loop. Because it understands it as a "shape," you can ask questions like:
* "What is the area of this shape?"
* "Is this shape a circle or a square?"
* "Where is the center point?"



### 3. The Golden Rule of Contours

To find contours, OpenCV needs a **Binary Image** (Black and White only).

* It assumes **Black is Background**.
* It assumes **White is Object**.
* It draws a line around the white stuff.

**In summary:** If you want to *see* the details, use Edge Detection. If you want to **count, measure, or recognize** objects, use Contours.